# Transfer Learning with VGG16 for Deep Space Object Classification





---


[**VGG16**](https://arxiv.org/pdf/1409.1556) is a convolutional neural network architecture with **16 layers** that are trainable (13 convolutional + 3 fully-connected). It was initially trained on the **[ImageNet](https://www.image-net.org/about.php)** dataset.


In Transfer Learning, we take these pre-trained weights, which learned general image features, and reuse them for a new classification task on our own dataset. This approach usually speeds up training and can improve accuracy when the new dataset is smaller or less diverse than ImageNet.

## 1. Add imports

We import **TensorFlow(2.18.0) / Keras** for building and training. We use **matplotlib** for plotting metrics and **numpy** for numerical operations. 'preprocess_input' from VGG16 is used later to normalize images.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt
import numpy as np
import os
print(tf.__version__)

# 3. Dataset

In this section, we will load our deep space objects dataset. It includes images from galaxies, nebulae, and star clusters. We will pull it from Google Drive, then split it into train, validation, and test sets. We will also apply basic preprocessing such as input resizing, normalization, etc.

## 3.1 Google Drive mounting


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3.2 Data splitting for Training and Validation

We first split our main dataset into training and validation subsets. This is done using the `validation_split` parameter in `ImageDataGenerator`, where 20% of the images from the dataset are reserved for validation.

In [ ]:
dataset_dir = '/content/drive/MyDrive/Transfer Learning in Astronomy/DATA_split/train'
split_datagen = ImageDataGenerator(validation_split=0.2)

train_generator = split_datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = split_datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


## 3.3 Data Preprocessing

Next, we apply preprocessing to the images so they match what VGG16 expects. The `preprocessing_function` parameter of the `ImageDataGenerator` lets us use the VGG16-specific `preprocess_input` function. This step normalizes the image pixel values (e.g., scaling and mean subtraction) to be consistent with the data on which VGG16 was originally trained.

In [ ]:
test_dataset_dir = '/content/drive/MyDrive/Transfer Learning in Astronomy/DATA_split/test'

# Create an ImageDataGenerator for the test set with the VGG16 preprocessing function
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Generator for the test set
test_generator = test_datagen.flow_from_directory(
    test_dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False,  # Important for evaluation to maintain order
    classes=list(train_generator.class_indices.keys())
)


# 4. Import VGG16 Model and Build the Classifier

## 4.1 Load and Inspect the VGG16 Architecture

`include_top=False` removes the default classifier, allowing us to add our own layers tailored to our classification task.

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.summary()


## 4.2 Freeze the Pre-Trained Layers

freezing the layers ensures that the pre-trained feature extraction remains intact during initial training.

In [ ]:
# Freeze the layers of the VGG16 base model to preserve the pre-trained weights
for layer in base_model.layers:
    layer.trainable = False


## 4.3 Build a New Classifier on Top

In [ ]:
# Build a new classifier on top of the frozen VGG16 base
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.summary()


# 5. Train the Model

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


# 6. Evaluate and Visualize the Model Performance

## 6.1 Plot Training Metrics

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()


## 6.2 Final Evaluation on the Test Set

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_acc}")


# 7. Predict a Specific Image

In [ ]:
from tensorflow.keras.preprocessing import image

img_path = 'img_path.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

predictions = model.predict(x)
predicted_class = np.argmax(predictions, axis=1)
print("Predicted Class Index:", predicted_class[0])